# Imports

In [1]:
import pandas as pd 
import geopandas as gpd
import warnings


warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.2f}'.format
pd.set_option("display.max_columns", None)

In [2]:
%cd ..
data = pd.read_csv('datasets/non_scaled_preprocessed_data.csv', index_col = 0)
data_geo = gpd.GeoDataFrame(data, geometry = gpd.points_from_xy(data['longitude'], data['latitude'], crs = '4236'))
%cd Geo-Analysis

c:\Users\luisb\OneDrive\Documentos\uni\Year 2\Semester 2\Machine learning 2\ML2_project
c:\Users\luisb\OneDrive\Documentos\uni\Year 2\Semester 2\Machine learning 2\ML2_project\Geo-Analysis


# Introduction

In this notebook, we will perform a geographical analysis of our customers, looking for different regional groups and then comparing their data to identify possible clusters.

We're gonna start by having a simple look at our geographical dataset.

In [3]:
data_geo

,kids_home,teens_home,number_complaints,distinct_stores_visited,pct_spend_groceries,pct_spend_electronics,typical_hour,pct_spend_vegetables,pct_spend_nonalcohol_drinks,pct_spend_alcohol_drinks,pct_spend_meat,pct_spend_fish,pct_spend_hygiene,pct_spend_videogames,pct_spend_petfood,lifetime_total_distinct_products,percentage_of_products_bought_promotion,Age,male,loyalty_card,total_spend,customer_tenure,customer_name,Bsc.,Hsc.,Msc.,Phd.,latitude,longitude,geometry
customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
39634,1,1,1,2,43.22,16.78,17.00,1.14,1.43,2.23,5.26,9.58,5.13,9.32,5.93,32.00,7.22,60.00,1.00,0.00,42359.00,14.00,David Frink,0,1,0,0,38.74,-9.15,POINT (-9.14907 38.73564)
43460,1,0,1,2,30.57,8.13,8.00,2.10,2.28,7.42,6.51,11.73,12.89,6.27,12.11,2566.00,20.62,36.00,0.00,0.00,16407.00,10.00,Belinda Terry,0,1,0,0,38.76,-9.16,POINT (-9.16080 38.76270)
23937,0,1,1,3,17.52,23.04,10.00,1.70,4.82,9.79,2.52,13.28,7.39,8.51,11.42,1282.00,13.62,55.00,0.00,1.00,17675.00,15.00,Evelyn Calico,0,1,0,0,38.72,-9.17,POINT (-9.16510 38.71743)
2431,0,0,0,1,67.15,1.28,17.00,0.95,1.56,0.37,1.77,0.94,0.33,0.00,25.65,70.00,21.59,83.00,0.00,1.00,16121.00,17.00,Ora Boggess,0,1,0,0,38.78,-9.15,POINT (-9.15032 38.78217)
10090,0,0,0,1,13.30,10.88,17.20,0.15,0.63,54.05,1.03,3.28,11.20,4.25,1.22,4.00,18.10,23.00,1.00,1.00,5241.00,0.00,Lonnie Hopson,0,1,0,0,38.77,-9.18,POINT (-9.17702 38.77040)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48487,1,1,0,1,24.10,21.09,17.00,2.27,2.48,2.14,5.26,12.67,5.86,14.70,9.43,14.00,23.33,41.00,0.00,0.00,29835.00,8.00,Amelia Crawford,0,1,0,0,38.76,-9.17,POINT (-9.16615 38.76069)
24856,1,1,1,1,28.05,28.67,12.00,1.50,1.39,3.75,6.51,9.72,4.05,9.56,6.80,25.00,6.10,32.00,0.00,1.00,40071.00,10.00,Silvia Ramos,0,1,0,0,38.76,-9.17,POINT (-9.16664 38.76028)
21373,0,0,0,1,31.98,1.27,12.00,1.53,2.22,0.23,1.75,0.75,0.94,0.00,59.34,44.00,9.77,30.00,0.00,0.00,18946.00,7.00,Addie Coronel,0,1,0,0,38.77,-9.18,POINT (-9.18155 38.76992)


In [4]:
# Estimating the best CRS for our data
data_geo.estimate_utm_crs()

<Projected CRS: EPSG:32629>
Name: WGS 84 / UTM zone 29N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 12°W and 6°W, northern hemisphere between equator and 84°N, onshore and offshore. Algeria. Côte D'Ivoire (Ivory Coast). Faroe Islands. Guinea. Ireland. Jan Mayen. Mali. Mauritania. Morocco. Portugal. Sierra Leone. Spain. United Kingdom (UK). Western Sahara.
- bounds: (-12.0, 0.0, -6.0, 84.0)
Coordinate Operation:
- name: UTM zone 29N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [5]:
# Converting our data the best CRS we found above
data_geo.to_crs(32629, inplace = True)

In [6]:
# Plotting our data on a map
# We are only using the first 1000 observations as this allows us to reduce the load 
# of our notebook while still getting the relevant information
# (We, of course, checked that the same conclusions can be reached)
data_geo.iloc[:1000].explore()

Now that we have the geographic distribution of our customers, we can clearly see that the large majority are in Lisbon, but we can also see two much smaller groups in Ericeira and Peniche. We are now going to split our customers in two groups, the ones from Lisbon and the ones outside of Lisbon, to do this we will find a random customer in Lisbon and establish a buffer around them.

In [7]:
# Firstly, we find a random customer in Lisbon to use as our reference point
ref = data_geo[data_geo.index == 26745]
ref.explore()

In [8]:
# Now using our reference plot, we'll plot a 20 Km radius around it to see 
# if this effectively splits our customers in the two groups mentioned above
base = ref.geometry.buffer(20000).explore(color = 'red')
data_geo.iloc[:1000].explore(m = base)

In [9]:
# Now that we know that a 20Km radius from our reference point
# performs the desired division of customers, we're going to
# actually divide the data into 2 datasets
Lisbon_geo = data_geo[data_geo.within(ref.iloc[0].geometry.buffer(20000))]
Not_Lisbon_geo = data_geo[~ data_geo.within(ref.iloc[0].geometry.buffer(20000))]

In [10]:
# Checking the number of customers outside Lisbon
len(Not_Lisbon_geo)

388

In [11]:
Not_Lisbon_geo

,kids_home,teens_home,number_complaints,distinct_stores_visited,pct_spend_groceries,pct_spend_electronics,typical_hour,pct_spend_vegetables,pct_spend_nonalcohol_drinks,pct_spend_alcohol_drinks,pct_spend_meat,pct_spend_fish,pct_spend_hygiene,pct_spend_videogames,pct_spend_petfood,lifetime_total_distinct_products,percentage_of_products_bought_promotion,Age,male,loyalty_card,total_spend,customer_tenure,customer_name,Bsc.,Hsc.,Msc.,Phd.,latitude,longitude,geometry
customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
48086,0,0,1,1,23.32,0.08,7.00,0.39,0.45,0.18,0.14,75.32,0.13,0.00,0.00,14.00,24.77,58.00,1.00,0.00,64304.00,13.00,Charles`s Fishy,0,1,0,0,39.36,-9.38,POINT (466557.125 4356544.053)
31838,0,0,1,1,22.86,0.17,9.00,0.53,0.74,0.06,0.05,75.42,0.17,0.00,0.00,24.00,0.00,61.00,0.00,1.00,60528.00,13.00,Ursula`s Fishy,0,1,0,0,38.96,-9.42,POINT (463218.408 4312897.817)
41519,0,0,0,1,27.14,0.31,9.00,0.21,0.52,0.15,0.16,71.23,0.27,0.00,0.00,33.00,39.93,68.00,0.00,0.00,54085.00,9.00,Lindsay`s Fishy,0,1,0,0,38.96,-9.42,POINT (463268.824 4312875.546)
41076,0,0,0,1,29.92,0.71,12.00,0.82,0.50,0.14,0.35,67.40,0.16,0.00,0.00,34.00,28.27,82.00,0.00,0.00,50499.00,14.00,Carol`s Fishy,0,1,0,0,39.36,-9.38,POINT (466529.503 4356535.696)
43120,0,0,1,1,29.64,0.35,9.00,0.55,0.61,0.19,0.13,68.49,0.03,0.00,0.00,20.00,39.62,44.00,0.00,0.00,57166.00,13.00,Rosetta`s Fishy,0,1,0,0,39.36,-9.38,POINT (466468.330 4356512.613)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24463,0,0,1,1,28.25,0.13,17.00,0.16,0.98,0.15,0.09,70.00,0.23,0.00,0.00,7.00,22.83,68.00,0.00,1.00,57980.00,10.00,Stephanie`s Fishy,0,1,0,0,39.36,-9.38,POINT (466479.403 4356612.160)
12487,0,0,0,1,29.26,0.29,17.00,0.69,0.46,0.24,0.18,68.77,0.12,0.00,0.00,29.00,31.52,51.00,1.00,1.00,59522.00,21.00,Derrick`s Fishy,0,1,0,0,38.96,-9.42,POINT (463229.987 4312833.307)
14727,0,0,1,1,25.78,0.29,9.00,0.94,0.71,0.17,0.10,72.77,0.18,0.00,0.00,14.00,36.52,39.00,1.00,1.00,55018.00,14.00,Tyler`s Fishy,0,1,0,0,38.96,-9.42,POINT (463218.834 4312978.672)


In [12]:
len(data[(data['customer_name'].str.contains('Fishy'))])/len(Not_Lisbon_geo)

1.0

In [13]:
len(data[(data['customer_name'].str.contains('Fishy') & (data_geo.index.isin(Not_Lisbon_geo.index)))])/len(Not_Lisbon_geo)

1.0

Now that we have split our data, we can see a clear pattern in the customers outside of Lisbon, they all have "Fishy" in their name. This indicates that we could potentially have a particular segment of customers outside of the Lisbon area. Furthermore, we can see that all of our customers that have "Fishy" in their name are from outisde of Lisbon. With all this in mind, a comparative analysis will be performed between the two groups, using the non-geographic dataset.

In [14]:
Lisbon = data[data.index.isin(Lisbon_geo.index)]
Not_Lisbon = data[~ data.index.isin(Lisbon_geo.index)]


In [15]:
Lisbon.describe().T

,count,mean,std,min,25%,50%,75%,max
kids_home,43243.00,1.09,1.24,0.00,0.00,1.00,1.00,9.00
teens_home,43243.00,0.86,0.84,0.00,0.00,1.00,1.00,6.00
number_complaints,43243.00,0.71,0.62,0.00,0.00,1.00,1.00,9.00
distinct_stores_visited,43243.00,1.67,1.03,1.00,1.00,1.00,2.00,8.00
pct_spend_groceries,43243.00,37.78,11.65,0.00,31.71,37.72,44.95,89.73
pct_spend_electronics,43243.00,10.58,9.11,-0.00,3.22,7.96,16.31,56.67
typical_hour,43243.00,13.55,4.50,6.00,10.00,13.00,17.00,24.00
pct_spend_vegetables,43243.00,5.24,5.80,0.00,1.43,2.89,5.40,41.39
pct_spend_nonalcohol_drinks,43243.00,4.22,3.01,0.00,2.26,3.64,5.30,43.65
pct_spend_alcohol_drinks,43243.00,4.54,6.50,0.00,1.60,3.00,5.31,71.28


In [16]:
Not_Lisbon.describe().T

,count,mean,std,min,25%,50%,75%,max
kids_home,388.00,0.02,0.15,0.00,0.00,0.00,0.00,1.00
teens_home,388.00,0.03,0.17,0.00,0.00,0.00,0.00,1.00
number_complaints,388.00,0.51,0.50,0.00,0.00,1.00,1.00,1.00
distinct_stores_visited,388.00,1.00,0.00,1.00,1.00,1.00,1.00,1.00
pct_spend_groceries,388.00,26.72,10.46,16.17,23.17,25.21,27.32,95.05
pct_spend_electronics,388.00,0.37,0.38,0.00,0.15,0.29,0.50,5.45
typical_hour,388.00,10.03,4.09,6.00,7.00,8.00,12.00,17.00
pct_spend_vegetables,388.00,0.78,0.62,0.01,0.37,0.66,1.06,6.50
pct_spend_nonalcohol_drinks,388.00,0.60,0.30,0.03,0.45,0.56,0.70,3.36
pct_spend_alcohol_drinks,388.00,0.19,0.12,0.00,0.12,0.18,0.24,0.81


We can right away see a clear differnce and an obvious pattern in the way the group outside of Lisbon buys products, it's clear
(by also looking at their name) that these customers buy a lot fish. They could potentially be fishermen buying bait, as all of their expenses in other areas are extremely low ruling out the option of these being restaurants for example. With all this in mind, we now have our first cluster 'Fishermen'. We will now update and re-export our cluster analysis dataset with a Final Cluster variable where this segment is already represented. We will also drop the latitude and longitude as those were only present to allow us to obtain a GeoDataFrame.

In [17]:
data.loc[data.index.isin(Not_Lisbon.index), 'Final_Cluster'] = 'Fishermen'

In [18]:
data['Final_Cluster'].value_counts()

Final_Cluster
Fishermen    388
Name: count, dtype: int64

In [19]:
data.fillna('Unspecified', inplace = True)

In [20]:
data.drop(['longitude', 'latitude'], axis = 1, inplace = True)

In [21]:
data

,kids_home,teens_home,number_complaints,distinct_stores_visited,pct_spend_groceries,pct_spend_electronics,typical_hour,pct_spend_vegetables,pct_spend_nonalcohol_drinks,pct_spend_alcohol_drinks,pct_spend_meat,pct_spend_fish,pct_spend_hygiene,pct_spend_videogames,pct_spend_petfood,lifetime_total_distinct_products,percentage_of_products_bought_promotion,Age,male,loyalty_card,total_spend,customer_tenure,customer_name,Bsc.,Hsc.,Msc.,Phd.,Final_Cluster
customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
39634,1,1,1,2,43.22,16.78,17.00,1.14,1.43,2.23,5.26,9.58,5.13,9.32,5.93,32.00,7.22,60.00,1.00,0.00,42359.00,14.00,David Frink,0,1,0,0,Unspecified
43460,1,0,1,2,30.57,8.13,8.00,2.10,2.28,7.42,6.51,11.73,12.89,6.27,12.11,2566.00,20.62,36.00,0.00,0.00,16407.00,10.00,Belinda Terry,0,1,0,0,Unspecified
23937,0,1,1,3,17.52,23.04,10.00,1.70,4.82,9.79,2.52,13.28,7.39,8.51,11.42,1282.00,13.62,55.00,0.00,1.00,17675.00,15.00,Evelyn Calico,0,1,0,0,Unspecified
2431,0,0,0,1,67.15,1.28,17.00,0.95,1.56,0.37,1.77,0.94,0.33,0.00,25.65,70.00,21.59,83.00,0.00,1.00,16121.00,17.00,Ora Boggess,0,1,0,0,Unspecified
10090,0,0,0,1,13.30,10.88,17.20,0.15,0.63,54.05,1.03,3.28,11.20,4.25,1.22,4.00,18.10,23.00,1.00,1.00,5241.00,0.00,Lonnie Hopson,0,1,0,0,Unspecified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48487,1,1,0,1,24.10,21.09,17.00,2.27,2.48,2.14,5.26,12.67,5.86,14.70,9.43,14.00,23.33,41.00,0.00,0.00,29835.00,8.00,Amelia Crawford,0,1,0,0,Unspecified
24856,1,1,1,1,28.05,28.67,12.00,1.50,1.39,3.75,6.51,9.72,4.05,9.56,6.80,25.00,6.10,32.00,0.00,1.00,40071.00,10.00,Silvia Ramos,0,1,0,0,Unspecified
21373,0,0,0,1,31.98,1.27,12.00,1.53,2.22,0.23,1.75,0.75,0.94,0.00,59.34,44.00,9.77,30.00,0.00,0.00,18946.00,7.00,Addie Coronel,0,1,0,0,Unspecified


In [22]:
%cd ..
data.to_csv('datasets/For_analysis_non_scaled_preprocessed_data.csv')

c:\Users\luisb\OneDrive\Documentos\uni\Year 2\Semester 2\Machine learning 2\ML2_project
